In [1]:
# Importation des bibliothèques
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
#Architecture du LeNet-5
class LeNet5_Improved(nn.Module):
    def __init__(self, num_classes=33, dropout_rate=0.5):
        super(LeNet5_Improved, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool1 = nn.AvgPool2d(2, 2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool2 = nn.AvgPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(120, 84)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = self.dropout1(F.relu(self.fc1(x)))
        x = self.dropout2(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x


In [3]:
#Chargement du dataset AMHCD
data_dir = "/amhcd-data-64/tifinagh-images"

train_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((32, 32)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transform)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=test_transform)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '/amhcd-data-64/tifinagh-images/train'

In [ ]:
# Entrainement, validation et test
def train(model, loader, optimizer, criterion, device):
    model.train()
    loss_total, correct = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        loss_total += loss.item()
        correct += (outputs.argmax(1) == y).sum().item()
    return loss_total / len(loader), correct / len(loader.dataset)

def evaluate(model, loader, criterion, device):
    model.eval()
    loss_total, correct = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss_total += criterion(outputs, y).item()
            preds = outputs.argmax(1)
            correct += (preds == y).sum().item()
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return loss_total / len(loader), correct / len(loader.dataset), y_true, y_pred


In [ ]:
# Boucle d’entraînement
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5_Improved().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(10):
    tl, ta = train(model, train_loader, optimizer, criterion, device)
    vl, va, _, _ = evaluate(model, val_loader, criterion, device)
    train_losses.append(tl)
    val_losses.append(vl)
    train_accs.append(ta)
    val_accs.append(va)
    print(f\"Epoch {epoch+1}: Train Acc={ta:.4f}, Val Acc={va:.4f}\")


In [ ]:
# Matrice de Confusion
_, _, y_true, y_pred = evaluate(model, test_loader, criterion, device)
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_data.classes)
disp.plot(xticks_rotation=90)
plt.title(\"Confusion Matrix on AMHCD Test Set\")
plt.show()


In [ ]:
# Visualisation des courbes de perte et d’exactitude
def plot_training_curves(train_acc, val_acc, train_loss, val_loss):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title("Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_loss, label='Train Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title("Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Appel
plot_training_curves(train_accs, val_accs, train_losses, val_losses)


In [ ]:
# Visualisation du "feature maps"
def visualize_feature_maps(model, device, image_tensor):
    model.eval()
    image_tensor = image_tensor.unsqueeze(0).to(device)  # Batch de 1
    with torch.no_grad():
        x = model.bn1(model.conv1(image_tensor))
        x = F.relu(x)

    feature_maps = x.squeeze().cpu().numpy()

    plt.figure(figsize=(12, 6))
    for i in range(min(6, feature_maps.shape[0])):  # Les 6 premiers filtres
        plt.subplot(2, 3, i + 1)
        plt.imshow(feature_maps[i], cmap='gray')
        plt.axis('off')
        plt.title(f'Feature Map {i+1}')
    plt.suptitle("Feature Maps - Convolution Layer 1")
    plt.tight_layout()
    plt.show()
